In [1]:
import pandas as pd
import io
import requests
import glob
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()
from bs4 import BeautifulSoup
import re
import calendar
import locale
import yfinance
# Set the locale to German
import pygsheets
import datawrapper

from anthropic import Anthropic
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')



In [2]:

def load_excel_from_url(url):
    """Loads an Excel file from a URL into a pandas DataFrame.

    Args:
        url: The URL of the Excel file.

    Returns:
        A pandas DataFrame containing the data from the Excel file, or None if
        an error occurs.
    """
    try:
        headers = {
               "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
           }
        response = requests.get(url, headers=headers, stream=True) # Make the request with headers
        response.raise_for_status()  # Raise an exception for bad status codes (like 403)

        # Read the content into a BytesIO object for pandas
        excel_data = io.BytesIO(response.content)

        return excel_data
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None


In [3]:
us_trade="https://www.census.gov/foreign-trade/statistics/country/index.html#:~:text=Download%20Full%20Dataset%20(XLSX%20%2D%200.1%20MB)"

us_stats=load_excel_from_url(us_trade)
us_stats

In [4]:
url="https://www.census.gov/foreign-trade/statistics/country/ctyseasonal.xlsx"


In [5]:

# Download the file using requests
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BytesIO object from the response content
    excel_file = io.BytesIO(response.content)
    
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file, engine='openpyxl')  # Specify engine if needed
    print(df)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

     year  cty_code               cty_desc          BJAN          BFEB  \
0    2009        20    All other countries -7.688157e+09 -6.061251e+09   
1    2009      3510                 Brazil  5.109941e+08  4.469206e+08   
2    2009        17               CAFTA-DR  3.034300e+08  2.818892e+08   
3    2009      1220                 Canada -1.790673e+09 -1.893448e+09   
4    2009      5700                  China -2.036187e+10 -1.900556e+10   
..    ...       ...                    ...           ...           ...   
371  2025         9  South/Central America  4.319924e+09  4.621780e+09   
372  2025      4419            Switzerland -2.299469e+10 -2.001357e+10   
373  2025      5830                 Taiwan -7.480248e+09 -8.955526e+09   
374  2025      4120         United Kingdom -6.731581e+08  3.245911e+09   
375  2025      5520                Vietnam -1.229118e+10 -1.249563e+10   

             BMAR          BAPR          BMAY          BJUN          BJUL  \
0   -6.221323e+09 -7.484819e+09 -5

In [6]:
df.columns

Index(['year', 'cty_code', 'cty_desc', 'BJAN', 'BFEB', 'BMAR', 'BAPR', 'BMAY',
       'BJUN', 'BJUL', 'BAUG', 'BSEP', 'BOCT', 'BNOV', 'BDEC', 'BQ1', 'BQ2',
       'BQ3', 'BQ4', 'EJAN', 'EFEB', 'EMAR', 'EAPR', 'EMAY', 'EJUN', 'EJUL',
       'EAUG', 'ESEP', 'EOCT', 'ENOV', 'EDEC', 'EQ1', 'EQ2', 'EQ3', 'EQ4',
       'IJAN', 'IFEB', 'IMAR', 'IAPR', 'IMAY', 'IJUN', 'IJUL', 'IAUG', 'ISEP',
       'IOCT', 'INOV', 'IDEC', 'IQ1', 'IQ2', 'IQ3', 'IQ4'],
      dtype='object')

In [7]:
df[df.cty_desc=="China"]

,year,cty_code,cty_desc,BJAN,BFEB,BMAR,BAPR,BMAY,BJUN,BJUL,...,IJUL,IAUG,ISEP,IOCT,INOV,IDEC,IQ1,IQ2,IQ3,IQ4
4,2009,5700,China,-2.036187e+10,-1.900556e+10,-2.037261e+10,-1.842612e+10,-1.840727e+10,-1.757020e+10,-1.826092e+10,...,2.372623e+10,2.377449e+10,2.462898e+10,2.558478e+10,2.595788e+10,2.721828e+10,7.435777e+10,7.112548e+10,7.212970e+10,7.876093e+10
22,2010,5700,China,-2.029652e+10,-2.070019e+10,-2.174464e+10,-2.117937e+10,-2.301374e+10,-2.482953e+10,-2.343214e+10,...,3.112880e+10,3.213780e+10,3.171575e+10,3.155222e+10,3.197177e+10,3.198457e+10,8.426605e+10,9.019568e+10,9.498235e+10,9.550855e+10
40,2012,5700,China,-2.545280e+10,-2.470782e+10,-2.812206e+10,-2.638355e+10,-2.488227e+10,-2.607179e+10,-2.649391e+10,...,3.568304e+10,3.459547e+10,3.561117e+10,3.547865e+10,3.662270e+10,3.691888e+10,1.053582e+11,1.053510e+11,1.058897e+11,1.090202e+11
58,2013,5700,China,-2.835521e+10,-2.886388e+10,-2.405719e+10,-2.508820e+10,-2.684325e+10,-2.615814e+10,-2.714464e+10,...,3.646774e+10,3.693936e+10,3.684751e+10,3.702828e+10,3.742276e+10,3.833821e+10,1.102530e+11,1.071331e+11,1.102546e+11,1.127893e+11
78,2014,5700,China,-2.650031e+10,-2.756293e+10,-2.735816e+10,-2.892931e+10,-2.803673e+10,-2.919684e+10,-2.848447e+10,...,3.859166e+10,3.891895e+10,4.017323e+10,4.052143e+10,4.050391e+10,4.010795e+10,1.134893e+11,1.161684e+11,1.176838e+11,1.211333e+11
98,2015,5700,China,-2.991712e+10,-2.692465e+10,-3.983263e+10,-2.889481e+10,-3.020138e+10,-2.975705e+10,-2.947784e+10,...,3.963844e+10,4.164453e+10,4.114561e+10,3.955797e+10,3.929929e+10,3.746784e+10,1.258558e+11,1.185922e+11,1.224286e+11,1.163251e+11
118,2016,5700,China,-2.979297e+10,-3.153861e+10,-2.762581e+10,-2.787307e+10,-2.868884e+10,-2.854071e+10,-2.901581e+10,...,3.886216e+10,3.912173e+10,3.826001e+10,3.910617e+10,3.895725e+10,3.895993e+10,1.157574e+11,1.133953e+11,1.162439e+11,1.170233e+11
139,2017,5700,China,-2.864976e+10,-3.005331e+10,-3.141909e+10,-3.312414e+10,-3.103078e+10,-3.153944e+10,-3.132774e+10,...,4.171225e+10,4.151103e+10,4.129575e+10,4.240641e+10,4.434193e+10,4.511047e+10,1.220411e+11,1.267461e+11,1.245190e+11,1.318588e+11
166,2018,5700,China,-3.516939e+10,-3.502248e+10,-3.348426e+10,-3.332531e+10,-3.322285e+10,-3.309945e+10,-3.388871e+10,...,4.433528e+10,4.384805e+10,4.600810e+10,4.483239e+10,4.291889e+10,4.581347e+10,1.382721e+11,1.324859e+11,1.341914e+11,1.335648e+11
193,2019,5700,China,-3.306362e+10,-3.061860e+10,-2.823665e+10,-2.991378e+10,-2.970940e+10,-2.972676e+10,-2.910568e+10,...,3.831595e+10,3.768683e+10,3.601067e+10,3.476634e+10,3.388197e+10,3.269290e+10,1.192146e+11,1.165414e+11,1.120135e+11,1.013412e+11


In [8]:
base_col=["year","cty_code","cty_desc"]
export_cols=[]
import_cols=[]
for col in df.columns:
    if col[0]=="E":
        export_cols.append(col)
    elif col[0]=="I":
        import_cols.append(col)
base_col

['year', 'cty_code', 'cty_desc']

In [9]:
base_export_cols=base_col+export_cols
base_import_cols=base_col+import_cols
base_import_cols

['year',
 'cty_code',
 'cty_desc',
 'IJAN',
 'IFEB',
 'IMAR',
 'IAPR',
 'IMAY',
 'IJUN',
 'IJUL',
 'IAUG',
 'ISEP',
 'IOCT',
 'INOV',
 'IDEC',
 'IQ1',
 'IQ2',
 'IQ3',
 'IQ4']

In [10]:
export_df=df[base_export_cols]
export_df

,year,cty_code,cty_desc,EJAN,EFEB,EMAR,EAPR,EMAY,EJUN,EJUL,EAUG,ESEP,EOCT,ENOV,EDEC,EQ1,EQ2,EQ3,EQ4
0,2009,20,All other countries,3.092714e+10,3.222777e+10,3.163669e+10,2.979531e+10,3.143728e+10,3.199560e+10,3.202405e+10,3.228728e+10,3.320518e+10,3.501005e+10,3.566379e+10,3.594004e+10,9.479160e+10,9.322820e+10,9.751652e+10,1.066139e+11
1,2009,3510,Brazil,2.310141e+09,2.109861e+09,1.955903e+09,1.938554e+09,2.024336e+09,2.075956e+09,2.122650e+09,2.094565e+09,2.247110e+09,2.364203e+09,2.303859e+09,2.548317e+09,6.375905e+09,6.038846e+09,6.464325e+09,7.216379e+09
2,2009,17,CAFTA-DR,1.594814e+09,1.655722e+09,1.585226e+09,1.413914e+09,1.544395e+09,1.701161e+09,1.627044e+09,1.669154e+09,1.683253e+09,1.745002e+09,1.868941e+09,1.855674e+09,4.835761e+09,4.659470e+09,4.979451e+09,5.469618e+09
3,2009,1220,Canada,1.607276e+10,1.658391e+10,1.551450e+10,1.558492e+10,1.555190e+10,1.559846e+10,1.750951e+10,1.793495e+10,1.800158e+10,1.796037e+10,1.903583e+10,1.930928e+10,4.817117e+10,4.673528e+10,5.344604e+10,5.630547e+10
4,2009,5700,China,4.529859e+09,4.900156e+09,5.187716e+09,5.489906e+09,5.423565e+09,5.808423e+09,5.465313e+09,5.697671e+09,6.215815e+09,6.313081e+09,6.972480e+09,7.492694e+09,1.461773e+10,1.672189e+10,1.737880e+10,2.077826e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2025,9,South/Central America,1.799277e+10,1.847082e+10,1.794769e+10,1.789054e+10,1.811507e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.441128e+10,NaN,NaN,NaN
372,2025,4419,Switzerland,1.854150e+09,2.509547e+09,3.539151e+09,9.118536e+09,6.480149e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.902847e+09,NaN,NaN,NaN
373,2025,5830,Taiwan,3.698952e+09,4.101151e+09,4.286813e+09,4.974215e+09,4.645937e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.208692e+10,NaN,NaN,NaN
374,2025,4120,United Kingdom,5.982091e+09,9.365912e+09,8.244228e+09,8.996384e+09,7.816502e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.359223e+10,NaN,NaN,NaN


In [11]:
df

,year,cty_code,cty_desc,BJAN,BFEB,BMAR,BAPR,BMAY,BJUN,BJUL,...,IJUL,IAUG,ISEP,IOCT,INOV,IDEC,IQ1,IQ2,IQ3,IQ4
0,2009,20,All other countries,-7.688157e+09,-6.061251e+09,-6.221323e+09,-7.484819e+09,-5.537787e+09,-6.201629e+09,-8.128920e+09,...,4.015297e+10,3.968960e+10,4.240630e+10,4.296826e+10,4.423691e+10,4.572743e+10,1.147623e+11,1.124524e+11,1.222489e+11,1.329326e+11
1,2009,3510,Brazil,5.109941e+08,4.469206e+08,3.447423e+08,5.177740e+08,4.190184e+08,4.636909e+08,4.404457e+08,...,1.682204e+09,1.613042e+09,1.690305e+09,1.610670e+09,1.983716e+09,1.778059e+09,5.073248e+09,4.638363e+09,4.985551e+09,5.372444e+09
2,2009,17,CAFTA-DR,3.034300e+08,2.818892e+08,2.177867e+08,6.088876e+07,1.325944e+08,1.921538e+08,-4.636460e+07,...,1.673409e+09,1.675474e+09,1.685764e+09,1.791865e+09,1.882657e+09,1.826492e+09,4.032656e+09,4.273833e+09,5.034646e+09,5.501014e+09
3,2009,1220,Canada,-1.790673e+09,-1.893448e+09,-1.384375e+09,-1.091626e+09,-8.162940e+08,-1.853868e+09,-2.322835e+09,...,1.983234e+10,1.945058e+10,1.978832e+10,2.060733e+10,2.105888e+10,2.177427e+10,5.323966e+10,5.049706e+10,5.907124e+10,6.344048e+10
4,2009,5700,China,-2.036187e+10,-1.900556e+10,-2.037261e+10,-1.842612e+10,-1.840727e+10,-1.757020e+10,-1.826092e+10,...,2.372623e+10,2.377449e+10,2.462898e+10,2.558478e+10,2.595788e+10,2.721828e+10,7.435777e+10,7.112548e+10,7.212970e+10,7.876093e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2025,9,South/Central America,4.319924e+09,4.621780e+09,2.991398e+09,3.294295e+09,3.347217e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.247818e+10,NaN,NaN,NaN
372,2025,4419,Switzerland,-2.299469e+10,-2.001357e+10,-1.540641e+10,3.538335e+09,3.288457e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.631752e+10,NaN,NaN,NaN
373,2025,5830,Taiwan,-7.480248e+09,-8.955526e+09,-8.915797e+09,-9.665720e+09,-1.149461e+10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.743849e+10,NaN,NaN,NaN
374,2025,4120,United Kingdom,-6.731581e+08,3.245911e+09,1.443552e+09,4.347961e+09,3.015399e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.957593e+10,NaN,NaN,NaN


In [12]:
import locale

# Set the locale to English (United States)
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [13]:
def df_for_country(cols,country):    
    df_country=df[df["cty_desc"]==country]
    filtered_entries = [entry for entry in cols if 'Q' not in entry ]
    df_long_country=pd.melt(df_country, id_vars=['year'], value_vars=filtered_entries)
    df_long_country["month"]=df_long_country.variable.str[1:]
    df_long_country["date"]=df_long_country["month"]+"-"+df_long_country["year"].astype(str)
    df_long_country["date"] = pd.to_datetime(df_long_country["date"], format="%b-%Y")
    df_long_country.sort_values(by="date", ascending=True, inplace=True)
    df_long_red=df_long_country[["date","value"]]
    

    return(df_long_red)

In [14]:
china_df_export=df_for_country(export_cols,"China")
china_df_export

,date,value
0,2009-01-01,4.529859e+09
16,2009-02-01,4.900156e+09
32,2009-03-01,5.187716e+09
48,2009-04-01,5.489906e+09
64,2009-05-01,5.423565e+09
...,...,...
127,2025-08-01,NaN
143,2025-09-01,NaN
159,2025-10-01,NaN
175,2025-11-01,NaN


In [15]:
country="China"

In [16]:
def comp_calc(country):
    locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')  # For Linux/Mac

    df_sorted_import=df_for_country(import_cols,country)
    df_sorted_export=df_for_country(export_cols,country)
    df_sorted_import_export=pd.merge(df_sorted_export,df_sorted_import, on="date", how="outer",suffixes=("_export", "_import"))
    df_sorted_import_export["diff"]=df_sorted_import_export["value_export"]-df_sorted_import_export["value_import"]
    last_years=df_sorted_import_export[df_sorted_import_export["date"]>"2021-01-01"]
    last_years=last_years.dropna().reset_index(drop=True)
    last_years
    comparison_df=last_years[(last_years["date"]>"01-01-2021")&(last_years["date"]<"01-01-2025")]
    under_trump=last_years[(last_years["date"]>="01-01-2025")]
    array=[]
    for index, row in under_trump.iloc[-3:].iterrows():
        print(f"Date: {row['date']}, Export: {row['value_export']}, Import: {row['value_import']}, Import: {row['diff']}")
        filtered_df = comparison_df[(comparison_df['date'].dt.month == row['date'].month) & (comparison_df['date'].dt.day == row['date'].day)]
        locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')

        month_name = calendar.month_name[row['date'].month]

        inner_array=[month_name]
        for col in filtered_df.columns[1:]:

            value_mean=filtered_df[col].mean()
            relative_perc=row[col]/value_mean
            diff_perc_=(relative_perc-1)*100
            inner_array.append(diff_perc_)

  

        array.append(inner_array)
    comp_df=pd.DataFrame(array,columns=["date","relative_export","relative_import","relative_diff"])
    comp_df=comp_df.T
    comp_df.columns=comp_df.iloc[0]
    comp_df=comp_df.iloc[1:-1]
    return last_years, comp_df


In [17]:
comp_objects=comp_calc("China")
comp_objects[1]

Date: 2025-03-01 00:00:00, Export: 10755982226.848888, Import: 34932154062.998, Import: -24176171836.149113
Date: 2025-04-01 00:00:00, Export: 8632270989.95277, Import: 28327550943.81032, Import: -19695279953.85755
Date: 2025-05-01 00:00:00, Export: 6917937862.311373, Import: 20885789833.558525, Import: -13967851971.247152


date,März,April,Mai
relative_export,-14.660652,-31.846129,-42.624377
relative_import,-19.445894,-30.277664,-45.710313


In [18]:
sh = gc.open('TrumpTradestats')

sheet_name = "ChinaUS"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)


#Get the data from the Sheet into python as DF
#read = wks.get_as_df()
#read
wks.set_dataframe(comp_objects[1].reset_index(),(0,0))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygsheets/worksheet.py:1431: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(nan)


In [19]:
comp_objects[0]

,date,value_export,value_import,diff
0,2021-02-01,1.029851e+10,3.887719e+10,-2.857867e+10
1,2021-03-01,1.238673e+10,4.645083e+10,-3.406410e+10
2,2021-04-01,1.264053e+10,4.231189e+10,-2.967136e+10
3,2021-05-01,1.262373e+10,3.899123e+10,-2.636750e+10
4,2021-06-01,1.245587e+10,3.955801e+10,-2.710215e+10
5,2021-07-01,1.224742e+10,3.839666e+10,-2.614924e+10
6,2021-08-01,1.178552e+10,3.998891e+10,-2.820339e+10
7,2021-09-01,1.176846e+10,4.208977e+10,-3.032132e+10
8,2021-10-01,1.453576e+10,4.382949e+10,-2.929374e+10
9,2021-11-01,1.394960e+10,4.547907e+10,-3.152947e+10


In [21]:
comp_objects=comp_calc("Germany")
comp_objects[1]

Date: 2025-03-01 00:00:00, Export: 7441988677.733637, Import: 14732247118.418917, Import: -7290258440.68528
Date: 2025-04-01 00:00:00, Export: 7287409846.724853, Import: 12729525654.31712, Import: -5442115807.592267
Date: 2025-05-01 00:00:00, Export: 6490153602.651165, Import: 13260789003.900543, Import: -6770635401.249378


date,März,April,Mai
relative_export,25.930298,22.603342,8.186817
relative_import,21.476707,3.424624,6.697436


In [22]:
sh = gc.open('TrumpTradestats')

sheet_name = "GermanyUS"  # Replace with the name of the sheet you want to select
wks = sh.worksheet_by_title(sheet_name)

#Get the data from the Sheet into python as DF
#read = wks.get_as_df()
#read
wks.set_dataframe(comp_objects[1].reset_index(),(0,0))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygsheets/worksheet.py:1431: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(nan)


In [23]:
comp_objects[0]

,date,value_export,value_import,diff
0,2021-02-01,5.287991e+09,1.059922e+10,-5.311227e+09
1,2021-03-01,5.101973e+09,1.044306e+10,-5.341090e+09
2,2021-04-01,5.494988e+09,1.064620e+10,-5.151207e+09
3,2021-05-01,5.414449e+09,1.111188e+10,-5.697426e+09
4,2021-06-01,5.880364e+09,1.194040e+10,-6.060033e+09
5,2021-07-01,5.436096e+09,1.101808e+10,-5.581981e+09
6,2021-08-01,5.632242e+09,1.187899e+10,-6.246749e+09
7,2021-09-01,5.304824e+09,1.122544e+10,-5.920612e+09
8,2021-10-01,5.679180e+09,1.136535e+10,-5.686170e+09
9,2021-11-01,5.560811e+09,1.179553e+10,-6.234720e+09


In [24]:
comp_objects[1].to_csv("germany_relative.csv",index=False)

In [25]:
df=pd.DataFrame()